In [1]:
import os, sys, yaml

# basic
import matplotlib.pyplot as plt
from PIL import Image

# torch
import torch
# modules
ROOT_DIR = None
with open('../cfg/project/default.yaml', 'r') as f: ROOT_DIR = yaml.load(f, Loader=yaml.FullLoader)['root_dir']
sys.path.insert(0, os.path.abspath(f"{ROOT_DIR}"))

from pl_jdt.data.datasets.penn_fundan import PennFudanDataset
from pl_jdt.utils.references.detection.engine import train_one_epoch, evaluate
import pl_jdt.utils.references.detection.utils as utils
import pl_jdt.utils.references.detection.transforms as T
# import pl_jdt.utils.transforms as T
from pl_jdt.models.mask_rcnn import MaskRCNN

# global
DATA_PATH = f'{ROOT_DIR}/data/PennFudanPed'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
img = Image.open(f"{DATA_PATH}/PNGImages/FudanPed00001.png").convert('RGB')
plt.imshow(img)
plt.show()
mask = Image.open(f"{DATA_PATH}/PedMasks/FudanPed00001_mask.png").convert('L')
# each mask instance has a different color, from zero to N, where
# N is the number of instances. In order to make visualization easier,
# let's adda color palette to the mask.
mask.putpalette([
    0, 0, 0, # black background
    255, 0, 0, # index 1 is red
    255, 255, 0, # index 2 is yellow
    255, 153, 0, # index 3 is orange
])
plt.imshow(mask)
plt.show()

In [3]:
dataset = PennFudanDataset(DATA_PATH)
dataset[0][1].keys()

dict_keys(['boxes', 'labels', 'masks', 'image_id', 'area', 'iscrowd'])

In [4]:
# def get_instance_segmentation_model(num_classes):
#     model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
#     in_features = model.roi_heads.box_predictor.cls_score.in_features
#     # replace the pre-trained head with a new one
#     model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

#     # now get the number of input features for the mask classifier
#     in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
#     hidden_layer = 256
#     # and replace the mask predictor with a new one
#     model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
#                                                        hidden_layer,
#                                                        num_classes)

#     return model

In [5]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [6]:
# use our dataset and defined transformations
dataset = PennFudanDataset(DATA_PATH, get_transform(train=True))
dataset_test = PennFudanDataset(DATA_PATH, get_transform(train=False))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

model_kwargs = {
    'num_classes': 2,
    'hidden_layer': 256
}
model = MaskRCNN(**model_kwargs)
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.1)

In [7]:
num_epochs = 10
print_freq = 10
for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=print_freq)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)

Epoch: [0]  [ 0/60]  eta: 0:00:54  lr: 0.000090  loss: 2.8105 (2.8105)  loss_classifier: 0.7481 (0.7481)  loss_box_reg: 0.3405 (0.3405)  loss_mask: 1.6637 (1.6637)  loss_objectness: 0.0557 (0.0557)  loss_rpn_box_reg: 0.0025 (0.0025)  time: 0.9016  data: 0.2346  max mem: 2163
Epoch: [0]  [10/60]  eta: 0:00:20  lr: 0.000936  loss: 1.3680 (1.7269)  loss_classifier: 0.5157 (0.4831)  loss_box_reg: 0.2915 (0.2987)  loss_mask: 0.7141 (0.9195)  loss_objectness: 0.0191 (0.0207)  loss_rpn_box_reg: 0.0045 (0.0048)  time: 0.4024  data: 0.0270  max mem: 3315
Epoch: [0]  [20/60]  eta: 0:00:14  lr: 0.001783  loss: 1.0000 (1.2311)  loss_classifier: 0.2228 (0.3358)  loss_box_reg: 0.2861 (0.2857)  loss_mask: 0.3202 (0.5877)  loss_objectness: 0.0118 (0.0175)  loss_rpn_box_reg: 0.0041 (0.0045)  time: 0.3411  data: 0.0066  max mem: 3315
Epoch: [0]  [30/60]  eta: 0:00:10  lr: 0.002629  loss: 0.5768 (1.0179)  loss_classifier: 0.0970 (0.2560)  loss_box_reg: 0.2760 (0.2882)  loss_mask: 0.1821 (0.4537)  loss_ob